In [52]:
import cv2
import numpy as np
import mediapipe as mp
import tensorflow as tf
from tensorflow.keras.models import load_model
from google.colab.patches import cv2_imshow  # Display images in Colab

# Load trained RNN model
#model = load_model("/content/my_rnn_model.keras")
model = joblib.load("/content/football_talent_model.pkl")
# Initialize MediaPipe Pose (Multi-Person Mode)
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, model_complexity=1, min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# Open video
video_path = "Test.mp4"
cap = cv2.VideoCapture(video_path)

# Get video properties
frame_width, frame_height = 640, 360  # Resize for speed
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Define output video file
output_path = "output_video1.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

# Dictionary to store sequences per person
sequences = {}
sequence_length = 10  # Number of frames in sequence
frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    if frame_count % 5 != 0:  # Process every 5th frame
        continue

    frame = cv2.resize(frame, (frame_width, frame_height))  # Resize frame
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = pose.process(rgb_frame)

    if result.pose_landmarks:
        h, w, _ = frame.shape  # Frame dimensions

        for idx, landmarks in enumerate([result.pose_landmarks]):  # Extend this to multi-person detection
            # Extract height dynamically
            try:
                height_cm = np.linalg.norm([landmarks.landmark[mp_pose.PoseLandmark.NOSE].y -
                                            landmarks.landmark[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX].y]) * h
            except:
                height_cm = 0  # Handle missing keypoints

            # Placeholder attributes (Replace with real data if available)
            sprint_speed = np.random.randint(60, 90)
            agility = np.random.randint(60, 90)
            dribbling = np.random.randint(60, 90)
            stamina = np.random.randint(60, 90)
            missing_feature = np.random.randint(60, 90)  # Example placeholder

            # Extracted Features (Ensure Correct Shape)
            extracted_features = [
                height_cm,  # 1
                75,  # Placeholder for weight
                85,  # Placeholder for rating
                90,  # Placeholder for potential
                sprint_speed,  # 5
                dribbling,  # 6
                agility,  # 7
                stamina,  # 8
                missing_feature  # **9th Feature (FIXED)**
            ]

            # Store sequence per person
            if idx not in sequences:
                sequences[idx] = []
            sequences[idx].append(extracted_features)

            if len(sequences[idx]) > sequence_length:
                sequences[idx].pop(0)

            # Predict when sequence is ready
            if len(sequences[idx]) == sequence_length:
                input_sequence = np.expand_dims(np.array(sequences[idx]), axis=0)  # Shape (1, 10, 9)
                prediction = model.predict(input_sequence)
                label = "Talented" if prediction > 0.5 else "Not Talented"

                # Draw pose landmarks
                mp_drawing.draw_landmarks(frame, landmarks, mp_pose.POSE_CONNECTIONS,
                                          landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 255), thickness=2, circle_radius=3),
                                          connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2))

                # Get nose landmark for label positioning
                nose = landmarks.landmark[mp_pose.PoseLandmark.NOSE]
                x, y = int(nose.x * w), int(nose.y * h)
                cv2.putText(frame, label, (x, max(50, y - 50)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Write frame to output video
    out.write(frame)

# Release resources
cap.release()
out.release()

print("✅ Processing complete! Output video saved as:", output_path)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 271ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━

In [53]:
# prompt: Download the video

from google.colab import files
files.download('output_video1.mp4')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>